<a href="https://colab.research.google.com/github/nicikess/hsg-nlp-course/blob/main/CodeBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports for project

In [19]:
import torch
import os
import numpy as np
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)
from tqdm import tqdm, trange

!pip install transformers
from transformers import (AdamW,RobertaConfig,RobertaForSequenceClassification,RobertaTokenizer)

!pip install utils
from utils import (processors)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 39.7 MB/s 
     |████████████████████████████████| 596 kB 67.9 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ImportError: ignored

Set directories and file names

In [8]:
data_dir = '/content/drive/MyDrive/NLP/data/'
train_file = 'train.txt'
dev_file = 'valid.txt'
model_type = 'roberta'
model_name_or_path = 'mircrosoft/codebert-base'
task_name = 'codesearch'
output_dir = '/content/drive/MyDrive/NLP/data/output/'

Set seed

In [9]:
# init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

Enable GPU

In [10]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

[LOG] notebook with cpu computation enabled


Set hyperparameters

In [14]:
batch_size = 32
learning_rate = 1e-5
num_training_epochs = 8
warmup_steps = 0
start_epoch = 0
gradient_accumulation_steps = 1
adam_epsilon = 1e-8

Train the model

In [15]:
def train(train_dataset, model, optimizer):
  train_sampler = RandomSampler(train_dataset)
  train_dataloader = DataLoader(train_dataset, sampler = train_sampler, batch_size = batch_size)

  global_step = 0.0
  tr_loss = 0.0
  best_acc = 0.0

  model.zero_grad()
  train_iterator = trange(start_epoch, num_training_epochs, desc="Epoch")

  model.train()

  for idx, _ in enumerate(train_iterator):
    tr_loss = 0.0
    for step, batch in enumerate(train_dataloader):
      batch = tuple(t.to(device) for t in batch)
      inputs = {'input_ids': batch[0],
                'attention_mask': batch[1]}
      outputs = model(**inputs)
      loss = outputs[0]

      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)

      tr_loss = loss.item()

      if (step + 1) % gradient_accumulation_steps == 0:
        optimizer.step()
        model.zero_grad()
        global_step += 1

Load data

In [16]:
file = os.path.join(data_dir, train_file)
features = torch.load(file)
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)

UnpicklingError: ignored

Load model

In [ ]:
config_class = 'RobertaConfig'
model_class = 'RobertaForSequenceClassification'
tokenizer_class = 'RobertaTokenizer'
  
config = config_class.from_pretrained(model_name_or_path, num_labels=num_labels, finetuning_task=args.task_name)
    if args.tokenizer_name:
        tokenizer_name = args.tokenizer_name
    elif args.model_name_or_path:
        tokenizer_name = 'roberta-base'
    tokenizer = tokenizer_class.from_pretrained(tokenizer_name, do_lower_case=args.do_lower_case)
    model = model_class.from_pretrained(args.model_name_or_path, from_tf=bool('.ckpt' in args.model_name_or_path),
                                        config=config)

    if args.local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab

    # Distributed and parallel training
    model.to(args.device)

Set optimizer with parameters

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
      'weight_decay': args.weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [18]:
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)

TypeError: ignored

In [ ]:
train(dataset, model, optimizer)